# **Projet de programmation mathématique avancée**
## **- Kidney exchange problem -**
### Anna Kerebel, Hiba Shaimed, Kefan Sun, Victor Klötzer, Vinh Nguyen

 *14 mai 2021*

---

1. bla
2. blabla

$\phantom{saut de ligne}$

Dans ce projet, on s'intéresse à blablabla.

In [9]:
# Pakages utilisés
using JuMP 
using Gurobi
using DelimitedFiles
using LightGraphs
using MetaGraphs
using NBInclude
const GUROBI_ENV = Gurobi.Env()
const ϵ = 0.00001

Academic license - for non-commercial use only - expires 2021-06-25


1.0e-5

In [41]:
# Importation des autres fonctions

@nbinclude("01_data.ipynb")
@nbinclude("02_node.ipynb")
@nbinclude("03_master.ipynb")
@nbinclude("04_subproblem.ipynb")
@nbinclude("05_branch_and_price.ipynb")
@nbinclude("06_MIP.ipynb")

solve_MIP (generic function with 2 methods)

In [37]:
# Présentation des données

#=
https://juliagraphs.org/LightGraphs.jl/v1.1/pathing.html#Cycle-Detection-1
=#

data_folder = string(join(split(@__DIR__, '\\')[1:end-1],'\\'),"\\data")
instance = "MD-00001-00000111"
wmd_file = joinpath(data_folder, join([instance, ".wmd"]))
dat_file = joinpath(data_folder, join([instance, ".dat"]))
global GRAPH = read_graph(wmd_file, dat_file)
global VERTICES = 1:nv(GRAPH)
global EDGES = [(e.src,e.dst) for e in edges(GRAPH)]
global L = 3
global WEIGHTS = Dict{Tuple{Int,Int},Float32}((e.src,e.dst) => get_prop(GRAPH, e, :weight) for e in edges(GRAPH));

# À revoir
println("Nombre de couples donneur/receveur          : $(nv(GRAPH))")
println("Nombre d'arcs                               : $(ne(GRAPH))")
println("Longueur maximale autorisée pour les cycles : $L")

********* Read instance MD-00001-00000111 *********
Nombre de couples donneur/receveur          : 128
Nombre d'arcs                               : 4108
Longueur maximale autorisée pour les cycles : 3


In [34]:
result_MIP = @timed solve_MIP()

Academic license - for non-commercial use only - expires 2021-06-25
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 512 rows, 555738 columns and 1659218 nonzeros
Model fingerprint: 0x50731169
Variable types: 0 continuous, 555738 integer (555738 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 321.0000000
Presolve removed 3 rows and 30948 columns (presolve time = 6s) ...
Presolve removed 3 rows and 30948 columns
Presolve time: 6.01s
Presolved: 509 rows, 524790 columns, 1566366 nonzeros
Found heuristic solution: objective 322.0000000
Variable types: 0 continuous, 524790 integer (524790 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4708000e+04   1.416700e+04   0.000000e+00

(value = (351.0, [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0  …  -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0]), time = 32.4645787, bytes = 1186105915, gctime = 1.1803606, gcstats = Base.GC_Diff(1186105915, 1858, 31, 19038385, 6466, 0, 1180360600, 6, 2))

In [43]:
global BRANCHING_VERSION = "v2"
global SP_VERSION = "v2" 
global BP_INIT_VERSION = "v1"
global SHOW_ADDED_CYCLES = false
result = @timed solve_BP()

Liste des noeuds qu'il reste à traiter : [1]
NOEUD n°1 :
 |  Meilleure solution réalisable (entière) de valeur 1.0 trouvée 
 |  Meilleure solution réalisable (entière) de valeur 3.0 trouvée 
 |  Meilleure solution réalisable (entière) de valeur 6.0 trouvée 
 |  Meilleure solution réalisable (entière) de valeur 9.0 trouvée 
 |  Meilleure solution réalisable (entière) de valeur 12.0 trouvée 
 |  Meilleure solution réalisable (entière) de valeur 15.0 trouvée 
 |  Meilleure solution réalisable (entière) de valeur 18.0 trouvée 
 |  Meilleure solution réalisable (entière) de valeur 21.0 trouvée 
 |  Meilleure solution réalisable (entière) de valeur 24.0 trouvée 
 |  Meilleure solution réalisable (entière) de valeur 27.0 trouvée 
 |  Meilleure solution réalisable (entière) de valeur 30.0 trouvée 
 |  Meilleure solution réalisable (entière) de valeur 33.0 trouvée 
 |  Meilleure solution réalisable (entière) de valeur 45.0 trouvée 
 |  Meilleure solution réalisable (entière) de valeur 48.0 trou

 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 6.0
 |                                               et la borne supérieure : 6.0
 |  Ce noeud est élagué car irréalisable ou car sa borne est non-prometteuse 
LB=66.0, UB=83.0 
Liste des noeuds qu'il reste à traiter : [20]
NOEUD n°20 :
 |  Meilleure solution réalisable (entière) de valeur 83.0 trouvée 
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 83.0
 |                                               et la borne supérieure : 83.0
 |  La solution relâchée optimale était entière
 |  Valeur réalisable 83.0 trouvée
LB=83.0, UB=83.0 


(value = nothing, time = 49.876642199, bytes = 3202950063, gctime = 0.700318, gcstats = Base.GC_Diff(3202950063, 8928, 0, 66073524, 34520, 0, 700318000, 28, 0))

In [39]:
global BRANCHING_VERSION = "v1"
global SP_VERSION = "v2" 
global BP_INIT_VERSION = "v1"
global SHOW_ADDED_CYCLES = false
result = @timed solve_BP()

Liste des noeuds qu'il reste à traiter : [1]
NOEUD n°1 :
 |  Meilleure solution réalisable (entière) de valeur 1.0 trouvée 
 |  Ajout du cycle [24, 32, 72, 24] 
 |  Meilleure solution réalisable (entière) de valeur 3.0 trouvée 
 |  Ajout du cycle [45, 99, 123, 45] 
 |  Meilleure solution réalisable (entière) de valeur 6.0 trouvée 
 |  Ajout du cycle [45, 99, 123, 45] 
 |  Ajout du cycle [24, 32, 72, 24] 
 |  Ajout du cycle [3, 85, 102, 3] 
 |  Meilleure solution réalisable (entière) de valeur 9.0 trouvée 
 |  Ajout du cycle [3, 85, 102, 3] 
 |  Ajout du cycle [16, 85, 102, 16] 
 |  Ajout du cycle [3, 19, 41, 3] 
 |  Meilleure solution réalisable (entière) de valeur 12.0 trouvée 
 |  Ajout du cycle [3, 19, 41, 3] 
 |  Ajout du cycle [16, 85, 102, 16] 
 |  Ajout du cycle [61, 85, 102, 61] 
 |  Ajout du cycle [9, 11, 83, 9] 
 |  Meilleure solution réalisable (entière) de valeur 15.0 trouvée 
 |  Ajout du cycle [9, 11, 83, 9] 
 |  Ajout du cycle [11, 120, 123, 11] 
 |  Ajout du cycle [99, 

 |  Ajout du cycle [48, 62, 48] 
 |  Ajout du cycle [57, 75, 102, 57] 
 |  Ajout du cycle [15, 52, 65, 15] 
 |  Ajout du cycle [87, 100, 102, 87] 
 |  Ajout du cycle [20, 85, 88, 20] 
 |  Ajout du cycle [68, 80, 99, 68] 
 |  Ajout du cycle [31, 97, 121, 31] 
 |  Ajout du cycle [21, 33, 47, 21] 
 |  Ajout du cycle [3, 29, 36, 3] 
 |  Ajout du cycle [40, 75, 40] 
 |  Ajout du cycle [45, 77, 116, 45] 
 |  Ajout du cycle [17, 95, 99, 17] 
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 83.00000000000003
 |                                               et la borne supérieure : 83.0
Deux noeuds sont créés en branchant sur x_(2,24)
LB=66.0, UB=83.0 
Liste des noeuds qu'il reste à traiter : [2, 3]
NOEUD n°3 :
 |  Ajout du cycle [2, 24, 124, 2] 
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 3.0
 |                                               et la borne supérieure : 3.0
 |  Ce noeud est élagué car irréalisable ou car sa borne est non-promet

NOEUD n°21 :
 |  Ajout du cycle [3, 40, 45, 3] 
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 3.0
 |                                               et la borne supérieure : 3.0
 |  Ce noeud est élagué car irréalisable ou car sa borne est non-prometteuse 
LB=66.0, UB=83.0 
Liste des noeuds qu'il reste à traiter : [20]
NOEUD n°20 :
 |  Ajout du cycle [72, 78, 114, 72] 
 |  Ajout du cycle [3, 123, 3] 
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 82.99999999999997
 |                                               et la borne supérieure : 83.0
Deux noeuds sont créés en branchant sur x_(8,61)
LB=66.0, UB=83.0 
Liste des noeuds qu'il reste à traiter : [22, 23]
NOEUD n°23 :
 |  Ajout du cycle [8, 61, 96, 8] 
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 3.0
 |                                               et la borne supérieure : 3.0
 |  Ce noeud est élagué car irréalisable ou car sa borne est non-prometteuse 
L

 |  Ajout du cycle [10, 25, 104, 10] 
 |  Ajout du cycle [88, 91, 88] 
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 83.00000000000001
 |                                               et la borne supérieure : 83.0
Deux noeuds sont créés en branchant sur x_(9,83)
LB=66.0, UB=83.0 
Liste des noeuds qu'il reste à traiter : [40, 41]
NOEUD n°41 :
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 3.0
 |                                               et la borne supérieure : 3.0
 |  Ce noeud est élagué car irréalisable ou car sa borne est non-prometteuse 
LB=66.0, UB=83.0 
Liste des noeuds qu'il reste à traiter : [40]
NOEUD n°40 :
 |  Ajout du cycle [11, 55, 83, 11] 
 |  Ajout du cycle [3, 45, 57, 3] 
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 83.0
 |                                               et la borne supérieure : 83.0
Deux noeuds sont créés en branchant sur x_(10,39)
LB=66.0, UB=83.0 
Liste des noeuds qu

NOEUD n°61 :
 |  Ajout du cycle [10, 16, 102, 10] 
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 3.0
 |                                               et la borne supérieure : 3.0
 |  Ce noeud est élagué car irréalisable ou car sa borne est non-prometteuse 
LB=66.0, UB=83.0 
Liste des noeuds qu'il reste à traiter : [60]
NOEUD n°60 :
 |  Ajout du cycle [3, 10, 84, 3] 
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 82.99999999999997
 |                                               et la borne supérieure : 83.0
Deux noeuds sont créés en branchant sur x_(8,120)
LB=66.0, UB=83.0 
Liste des noeuds qu'il reste à traiter : [62, 63]
NOEUD n°63 :
 |  Ajout du cycle [8, 92, 120, 8] 
 |  Ajout du cycle [8, 92, 120, 8] 
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 3.0
 |                                               et la borne supérieure : 3.0
 |  Ce noeud est élagué car irréalisable ou car sa borne est non-promette

...

NOEUD n°242 :
 |  Ajout du cycle [1, 11, 116, 1] 
 |  Ajout du cycle [32, 91, 101, 32] 
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 83.0
 |                                               et la borne supérieure : 83.0
Deux noeuds sont créés en branchant sur x_(1,116)
LB=66.0, UB=83.0 
Liste des noeuds qu'il reste à traiter : [244, 245]
NOEUD n°245 :
 |  Ajout du cycle [1, 11, 116, 1] 
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 3.0
 |                                               et la borne supérieure : 3.0
 |  Ce noeud est élagué car irréalisable ou car sa borne est non-prometteuse 
LB=66.0, UB=83.0 
Liste des noeuds qu'il reste à traiter : [244]
NOEUD n°244 :
 |  Ajout du cycle [6, 11, 36, 6] 
 |  Ajout du cycle [3, 114, 123, 3] 
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 83.00000000000003
 |                                               et la borne supérieure : 83.0
Deux noeuds sont créés en br

 |                                               et la borne supérieure : 3.0
 |  Ce noeud est élagué car irréalisable ou car sa borne est non-prometteuse 
LB=66.0, UB=83.0 
Liste des noeuds qu'il reste à traiter : [262]
NOEUD n°262 :
 |  Ajout du cycle [16, 45, 49, 16] 
 |  Ajout du cycle [24, 71, 24] 
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 83.00000000000001
 |                                               et la borne supérieure : 83.0
Deux noeuds sont créés en branchant sur x_(2,99)
LB=66.0, UB=83.0 
Liste des noeuds qu'il reste à traiter : [264, 265]
NOEUD n°265 :
 |  Ajout du cycle [2, 99, 126, 2] 
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 3.0
 |                                               et la borne supérieure : 3.0
 |  Ce noeud est élagué car irréalisable ou car sa borne est non-prometteuse 
LB=66.0, UB=83.0 
Liste des noeuds qu'il reste à traiter : [264]
NOEUD n°264 :
 |  Ajout du cycle [45, 76, 122, 45] 
 |  

 |  Ajout du cycle [2, 11, 83, 2] 
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 83.00000000000001
 |                                               et la borne supérieure : 83.0
Deux noeuds sont créés en branchant sur x_(2,83)
LB=66.0, UB=83.0 
Liste des noeuds qu'il reste à traiter : [284, 285]
NOEUD n°285 :
 |  Ajout du cycle [2, 11, 83, 2] 
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 3.0
 |                                               et la borne supérieure : 3.0
 |  Ce noeud est élagué car irréalisable ou car sa borne est non-prometteuse 
LB=66.0, UB=83.0 
Liste des noeuds qu'il reste à traiter : [284]
NOEUD n°284 :
 |  Ajout du cycle [11, 74, 106, 11] 
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 83.0
 |                                               et la borne supérieure : 83.0
Deux noeuds sont créés en branchant sur x_(11,106)
LB=66.0, UB=83.0 
Liste des noeuds qu'il reste à traiter : [286, 2

NOEUD n°302 :
 |  Ajout du cycle [14, 16, 110, 14] 
 |  Ajout du cycle [27, 45, 107, 27] 
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 83.0
 |                                               et la borne supérieure : 83.0
Deux noeuds sont créés en branchant sur x_(11,80)
LB=66.0, UB=83.0 
Liste des noeuds qu'il reste à traiter : [304, 305]
NOEUD n°305 :
 |  Ajout du cycle [11, 80, 123, 11] 
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 3.0
 |                                               et la borne supérieure : 3.0
 |  Ce noeud est élagué car irréalisable ou car sa borne est non-prometteuse 
LB=66.0, UB=83.0 
Liste des noeuds qu'il reste à traiter : [304]
NOEUD n°304 :
 |  Relaxation du noeud résolue à l'optimalité avec la borne inférieure : 83.0
 |                                               et la borne supérieure : 83.0
Deux noeuds sont créés en branchant sur x_(16,102)
LB=66.0, UB=83.0 
Liste des noeuds qu'il reste à traiter :

LoadError: InterruptException:

In [143]:
println("Pour mettre le texte en couleur écrire \\e[ijm où i=3 ou i=9 pour l'intensité de la couleur j.")
println("(ne pas oublier de remettre \\e[00m en fin de ligne pour que la suite soit en noir)")
println(" i\\j  _0   _1   _2   _3   _4   _5   _6   _7   _8   _9")
println(" \e[00m3_  \e[30mabcd \e[31mabcd \e[32mabcd \e[33mabcd \e[34mabcd \e[35mabcd \e[36mabcd \e[37mabcd \e[38mabcd \e[39mabcd")
println(" \e[00m9_  \e[90mabcd \e[91mabcd \e[92mabcd \e[93mabcd \e[94mabcd \e[95mabcd \e[96mabcd \e[97mabcd \e[98mabcd \e[99mabcd")
print("truc \e[00mtruc")

Pour mettre le texte en couleur écrire \e[ijm où i=3 ou i=9 pour l'intensité de la couleur j.
(ne pas oublier de remettre \e en fin de ligne pour que la suite soit en noir)
 i\j  _0   _1   _2   _3   _4   _5   _6   _7   _8   _9
 3_  abcd abcd abcd abcd abcd abcd abcd abcd abcd abcd
 9_  abcd abcd abcd abcd abcd abcd abcd abcd abcd abcd
truc truc

Problème maître restreint ($\mathcal C_L^R \subset  \mathcal C_L$) :
$$\begin{array}{rlccrlc}
\max        & \displaystyle\sum\limits_{c\in \mathcal C_L^R} z_c w_c  & & \iff &
\max        & \displaystyle\sum\limits_{c\in \mathcal C_L^R} z_c \sum_{(i,j) \in E}\bar x_{(i,j)}^c w_{(i,j)} \\
\text{sous} & \displaystyle\sum\limits_{c\in \mathcal C_L^R \,|\, i\in c} z_c \le 1 & \forall i\in V & &
\text{sous} & \displaystyle\sum\limits_{c\in \mathcal C_L^R} z_c \,\bar y_i^c \le 1 & \forall i\in V ~~~\big\vert~~(\pi_i)_{i\in V}\\
            & z \in [0,1]^{|\mathcal C_L^R|} & & & & z \in [0,1]^{|\mathcal C_L^R|}
\end{array}$$

où :
*  $\bar x_{(i,j)}^c = 1$ si l'arc $(i,j)$ se trouve dans le cycle $c$, et $\bar x_{(i,j)}^c = 0$ sinon
*  $\bar y_i^c = 1$ si le sommet $i$ fait partie du cycle $c$, et $\bar y_i^c = 0$ sinon

$~$  

Sous-problème :
$$\begin{array}{rlcrlc}
\max        & \displaystyle\sum\limits_{(i,j)\in E} x_{(i,j)}^c w_{(i,j)} - \sum\limits_{i\in V} \pi_i^* y_i^c & \iff &
\max        & \displaystyle\sum\limits_{(i,j)\in E} x_{(i,j)} w_{(i,j)} - \sum\limits_{i\in V} \pi_i^* y_i \\
\text{sous} & c\in\mathcal C_L & &
\text{sous} & \displaystyle\sum\limits_{(i,j) \in\delta^-(i)} x_{(i,j)} = y_i & \forall i\in V\\
      & & & & \displaystyle\sum\limits_{(j,i) \in\delta^+(i)} x_{(j,i)} = y_i & \forall i\in V\\
      & & & & \displaystyle\sum\limits_{(i,j) \in E} x_{(i,j)} \le L \\
      & & & & \displaystyle\sum\limits_{(i,j) \in E} x_{(i,j)} \ge 2 \\
      & & & & x \in \{0,1\}^{|E|}, y \in \{0,1\}^{|V|}
\end{array}$$

$~$  

Relaxation lagrangienne (avec $\lambda_i \ge 0$, $\forall i\in V$) :
$$\begin{array}{rlcrl}
\max        & \displaystyle\sum\limits_{c\in \mathcal C_L} z_c \big(\sum_{(i,j) \in E} \bar x_{(i,j)}^c w_{(i,j)}\big) + \sum\limits_{i\in V}\lambda_i \big(1 - \sum\limits_{c\in \mathcal C_L} z_c \, \bar y_i^c \big)\\
\text{sous} & z \in [0,1]^{|\mathcal C_L|}
\end{array}$$

$$\begin{aligned}\displaystyle\underset{z}{\max}~\sum\limits_{c\in \mathcal C_L} z_c \big(\sum_{(i,j) \in E} \bar x_{(i,j)}^c w_{(i,j)}\big) &+ \sum\limits_{i\in V}\lambda_i \big(1 - \sum\limits_{c\in \mathcal C_L} z_c \, \bar y_i^c \big)\\
~~~~~ &= \underset{z}{\max}~\sum\limits_{i\in V}\lambda_i + \sum\limits_{c\in \mathcal C_L} z_c \big(\sum_{(i,j) \in E} \bar x_{(i,j)}^c w_{(i,j)} - \sum\limits_{i\in V} \bar y_i^c \big)\\
~~~~~ &\le \underset{z}{\max}~\sum\limits_{i\in V}\lambda_i + {\bf u}_\text{SP}^*\sum\limits_{c\in \mathcal C_L} z_c \\
~~~~~ &\le \underset{z}{\max}~\sum\limits_{i\in V}\lambda_i + {\bf u}_\text{SP}^*\times M
\end{aligned}$$


où $\displaystyle{\bf u}_\text{SP}^* := \underset{c\in\mathcal C_L}{\max}~\sum\limits_{(i,j)\in E} x_{(i,j)}^c w_{(i,j)} - \sum\limits_{i\in V} \pi_i^* y_i^c~~$ (solution optimale du sous-problème)

et $M$ est un majorant de $\displaystyle\sum\limits_{c\in \mathcal C_L} z_c$ le nombre de cycles nécessaires finalement (on peut prendre $M=\frac{|V|}2$).

$$\sum\limits_{i\in V}\pi_i^* + M\times {\bf u}_\text{SP}^*$$

# Tests

In [106]:
x = Dict{String,Any}()
x["vertices"] = [1,0,1,1,0,0,0,0,0,0,0]
x["oneedges"] = [(1,3),(4,3),(2,3)]
for (i,j) in x["oneedges"]
    println("$i $j")
end

1 3
4 3
2 3


In [121]:
x["oneedges"]

3-element Array{Tuple{Int64,Int64},1}:
 (1, 3)
 (4, 3)
 (2, 3)